# 0. 기본 정보
* 코드 작성자: 여서연
* 코드 작성일: 2024-05-23

# 1. 기초 설정

## 사용 라이브러리

In [1]:
import requests
import json
import xmltodict
import pandas as pd

from dotenv import load_dotenv
import os

## 기타 설정

In [2]:
# 데이터 프레임 열 전부 표시
pd.set_option('display.max_columns', None)

In [3]:
load_dotenv()
myKEY = os.environ.get('PUBLIC_DATA_API_KEY')

# 2. API 데이터 수집

## A. [환경부 국립환경과학원_골프장 농약사용량 DB](https://www.data.go.kr/data/15000937/openapi.do)

In [4]:
# 2020년 전국 데이터
golf_url = 'http://apis.data.go.kr/1480523/SgisGp/getSgisGpList1'
golf_params = {'serviceKey' : myKEY, 'pageNo' : '1', 'numOfRows' : '1000', 'resultType' : 'JSON', 'search_year': '2020'}

golf_response = requests.get(golf_url, params=golf_params)
#golf_response.content

In [5]:
golf_result = json.loads(golf_response.content)
#golf_result['getSgisGpList']['header']

In [6]:
golf = pd.DataFrame(golf_result['getSgisGpList']['item'])
golf.sample()

,SPOT_STD_CODE,YEAR,GOLFCLUB_NAME,ADDRESS,HOLE_CNT,SIDO,REG_YEAR,GOLF_TYPE_NAME,TOTAL_GROUND,PES_GROUND,NONE_GROUND,SPRAY_SUPPLY,REAL_SUPPLY,HA_REAL,HA_SPRAY,THA_REAL,THA_SPRAY
146,46880G040200002,2020,푸른솔GC,전라남도 장성군 동화면 남산리 874,27,전라남도,2009,대중제,1312986,842376,470610,3034.1000000000004,888.3880799999998,10.546217840964127,36.01835759803224,6.76616567122574,23.108395672154924


In [7]:
new_columns = {
    'SPOT_STD_CODE': '지점표준코드',
    'SIDO': '시도',
    'GOLFCLUB_NAME': '골프장명',
    'GOLF_TYPE_NAME': '골프장 형태',
    'ADDRESS': '소재지',
    'HOLE_CNT': '홀수',
    'REG_YEAR': '개장년도',
    'YEAR': '입력년도',
    'PES_GROUND': '농약사용면적(m2)',
    'TOTAL_GROUND': '면적총계(m2)',
    'NONE_GROUND': '농약미사용면적(m2)',
    'SPRAY_SUPPLY': '총사용량_실물량(kg)',
    'REAL_SUPPLY': '총사용량_성분량(kg)',
    'THA_SPRAY': '전체면적대비_ha_실물량(kg)',
    'THA_REAL': '전체면적대비_ha_성분량(kg)',
    'HA_SPRAY': '농약사용_ha_실물량(kg)',
    'HA_REAL': '농약사용_ha_성분량(kg)'
}

In [8]:
# 컬럼 이름 변경
golf.rename(columns=new_columns, inplace=True)
golf.sample()

,지점표준코드,입력년도,골프장명,소재지,홀수,시도,개장년도,골프장 형태,면적총계(m2),농약사용면적(m2),농약미사용면적(m2),총사용량_실물량(kg),총사용량_성분량(kg),농약사용_ha_성분량(kg),농약사용_ha_실물량(kg),전체면적대비_ha_성분량(kg),전체면적대비_ha_실물량(kg)
270,41630G040100002,2020,송추 컨트리클럽,경기도 양주시 광적면 비암리 산 23-1,18,경기도,1995,회원제,1401964.5,604942.5,797021.999999999,782.2300000000001,242.75607599999998,4.012878513247126,12.93065043371891,1.7315422466118078,5.57952786964292


In [9]:
# 시군구 추출 함수
def extract_sigungoo(address):
    parts = address.split()
    if len(parts) > 1:
        return parts[1]
    return None

# 시군구 컬럼 추가
golf['시군구'] = golf['소재지'].apply(extract_sigungoo)

# 시도 컬럼 바로 옆에 시군구 컬럼 배치
columns = list(golf.columns)
sido_index = columns.index('시도')
columns.insert(sido_index + 1, columns.pop(columns.index('시군구')))
golf = golf[columns]

In [10]:
golf.sample()

,지점표준코드,입력년도,골프장명,소재지,홀수,시도,시군구,개장년도,골프장 형태,면적총계(m2),농약사용면적(m2),농약미사용면적(m2),총사용량_실물량(kg),총사용량_성분량(kg),농약사용_ha_성분량(kg),농약사용_ha_실물량(kg),전체면적대비_ha_성분량(kg),전체면적대비_ha_실물량(kg)
166,47250G040200001,2020,뉴스프링빌2,경상북도 상주시 모서면 호음리 669,18,경상북도,상주시,2008,대중제,1098906,704844,394062,1122.25,394.18705,5.59254317267367,15.921962874054403,3.5870861565957415,10.21242945256464


In [11]:
golf.to_csv('../data/golf.csv', index=False, encoding='utf-8-sig')

In [12]:
unique_addresses = golf.groupby('시도')['소재지'].nunique().reset_index()
unique_addresses

,시도,소재지
0,강원도,53
1,경기도,145
2,경상남도,40
3,경상북도,50
4,광주광역시,4
5,대구광역시,4
6,대전광역시,4
7,부산광역시,12
8,서울특별시,2
9,세종특별자치시도,3


In [13]:
golf[golf['시도']=='강원도']['시군구'].unique()

array(['평창군', '횡성군', '원주시', '춘천시', '홍천군', '고성군', '삼척시', '정선군', '철원군',
       '강릉시', '양양군', '동해시', '영월군', '속초시', '태백시'], dtype=object)

In [14]:
golf[golf['시도']=='강원도'].to_csv('../data/golf_Gangwon.csv', index=False, encoding='utf-8-sig')

## B. [강원특별자치도_강수량 통계](https://www.data.go.kr/data/3081221/fileData.do)

In [15]:
rain_2020 = pd.read_csv('../data/강수량_20240526105254.csv', encoding='cp949', header=1)
rain_2019 = pd.read_csv('../data/강수량_20240526105308.csv', encoding='cp949', header=1)
rain_2018 = pd.read_csv('../data/강수량_20240526105317.csv', encoding='cp949', header=1)

In [16]:
rain_2020.head()

,시군별,전체,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,전체,1590.3,91.5,53.7,28.0,33.3,127.0,121.2,269.1,516.9,302.6,7.0,36.4,3.4
1,춘천시,1550.9,80.0,54.2,13.8,14.4,118.3,75.8,174.3,773.7,157.2,0.1,84.2,4.9
2,원주시,1317.0,58.7,65.5,21.2,15.7,80.0,49.6,204.7,634.0,148.9,11.3,23.6,3.8
3,강릉시,1812.9,107.5,77.5,47.1,65.6,95.5,270.4,375.9,150.4,613.8,0.6,7.7,0.9
4,동해시,1322.4,97.9,73.7,48.3,50.9,90.3,120.4,346.1,171.9,304.0,5.1,11.5,2.3


In [17]:
rain_2019.head()

,시군별,전체,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,전체,1048.3,3.4,21.3,40.7,56.9,17.3,103.7,200.0,172.1,207.0,148.5,68.0,9.6
1,춘천시,923.2,1.3,29.0,38.7,49.4,27.1,76.4,238.6,192.3,142.6,50.3,63.1,14.4
2,원주시,771.9,0.7,25.4,42.8,45.7,10.4,67.9,137.1,142.4,171.1,40.3,74.0,14.1
3,강릉시,1670.5,16.6,17.7,49.6,85.5,4.5,132.2,220.8,264.5,233.3,536.0,104.7,5.1
4,동해시,1532.8,14.3,15.4,51.5,102.6,3.3,125.4,191.4,149.9,255.6,512.5,102.1,8.8


In [18]:
rain_2018.head()

,시군별,전체,1월,2월,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
0,전체,1502.1,5.3,35.4,58.7,133.4,170.6,78.4,223.2,399.9,124.9,173.0,80.8,18.6
1,춘천시,1401.5,5.9,36.5,32.8,139.5,201.9,104.1,213.6,389.5,69.9,127.5,63.5,16.8
2,원주시,1229.2,4.4,27.4,53.6,112.6,211.3,131.8,122.5,258.2,147.6,89.5,45.0,25.3
3,강릉시,1599.4,2.5,52.0,94.0,151.4,162.9,38.6,195.5,419.1,161.2,248.0,51.3,22.9
4,동해시,1436.5,4.4,53.4,95.4,131.8,63.6,36.6,286.0,219.8,205.6,273.2,44.5,22.2


In [19]:
url = 'http://openapi.foodsafetykorea.go.kr/api/d2e4398d78ab41b1809c/I2400/json/1/1000'
response = requests.get(url)
result = json.loads(response.content)
data = pd.DataFrame(result['I2400']['row'])
data['YR'].unique()

array(['2012'], dtype=object)

## C. [환경부 국립환경과학원_수질 DB](https://www.data.go.kr/data/15081073/openapi.do)

### 시도별 수질측정망 지점

In [20]:
code = pd.read_excel('../API documents/국립환경과학원_openAPI_활용가이드_수질DB/물환경 수질측정망 운영결과 DB_물환경_코드_코드명.xlsx', header=1)
code

,구분,지점코드,지점명,조사기관,주소,대권역,중권역
0,하천수,1001A05,송천1,원주지방환경청,강원도 평창군 대관령면 용산리(용산교),한강,남한강상류
1,하천수,1001A10,송천4,원주지방환경청,강원도 정선군 여량면 유천리(송천교),한강,남한강상류
2,하천수,1001A15,골지천2,한국수자원공사,강원도 태백시 하사미동 290-1(하사미교),한강,남한강상류
3,하천수,1001A20,번천,한국수자원공사,강원도 삼척시 하장면 숙암리 90-4,한강,남한강상류
4,하천수,1001A25,골지천4,원주지방환경청,강원도 정선군 여량면 여량리(여량1교),한강,남한강상류
...,...,...,...,...,...,...,...
2213,하천수,5101A05,탐진강1(탐진A1),영산강물환경연구소,전라남도 장흥군 유치면 운월리(운월교),영산강ㆍ섬진강,탐진강
2214,하천수,5101A15,유치천1(탐진A2),영산강물환경연구소,전라남도 영암군 금정면 연소리(한대교),영산강ㆍ섬진강,탐진강
2215,하천수,5101A25,옴천천2(탐진A3),영산강물환경연구소,전라남도 강진군 옴천면 봉림리(봉림교),영산강ㆍ섬진강,탐진강
2216,하천수,5101A45,금강천1(탐진B1),영산강물환경연구소,전라남도 강진군 병영면 삭양리(삭양교),영산강ㆍ섬진강,탐진강


In [21]:
# 주소 컬럼의 데이터를 공백을 기준으로 분리하여 시도, 시군구 추출
code['시도'] = code['주소'].apply(lambda x: x.split()[0])
code['시군구'] = code['주소'].apply(lambda x: x.split()[1])

print(code['시도'].unique())
print()
print(code['시군구'].unique())

['강원도' '충청북도' '경기도' '서울특별시' '인천광역시' '충청남도' '인천광역시시' '경상북도' '경북' '대구광역시'
 '대구광역시시' '경상남도' '전라북도' '부산광역시' '울산광역시' '울산광역시시' '부산광역시시' '대전광역시' '대전광역시시'
 '세종특별자치시' '전라남도' '광주' '광주광역시' '제주특별자치도']

['평창군' '정선군' '태백시' '삼척시' '영월군' '횡성군' '원주시' '단양군' '제천시' '충주시' '청주시' '괴산군'
 '보은군' '음성군' '홍천군' '안성시' '이천시' '여주시' '양평군' '용인시' '화천군' '양구군' '춘천시' '인제군'
 '가평군' '남양주시' '광주시' '강동구' '성남시' '강남구' '성동구' '의정부시' '노원구' '성북구' '동대문구' '중구'
 '용산구' '영등포구' '마포구' '군포시' '안양시' '구로구' '동작구' '양천구' '강서구' '고양시' '김포시' '계양구'
 '파주시' '연천군' '철원군' '포천시' '양주시' '동두천시' '하남시' '구리시' '송파구' '천안시' '평택시' '화성시'
 '오산시' '의왕시' '수원시' '아산시' '강화군' '서구' '시흥시' '남동구' '화성군' '안산시' '고성군' '양양군'
 '속초시' '강릉시' '동해시' '봉화군' '안동시' '영양군' '청송군' '예천군' '의성군' '영주시' '문경시' '상주시'
 '군위군' '구미시' '김천시' '칠곡군' '성주군' '달성군' '영천시' '경산시' '동구' '북구' '달서구' '수성구'
 '합천군' '고령군' '창녕군' '달성구' '거창군' '의령군' '함양군' '남원시' '산청군' '진주시' '사천시' '하동군'
 '함안군' '창원시' '밀양시' '김해시' '경주시' '청도군' '양산시' '사상구' '사하구' '포항시' '울주군' '포항'
 '남구' '금정구' '수영구' '기장군' '동래구' '해운대구광역시' '부산광역시진구' '울진군' '영덕군' '남해군' '거제시'
 '창원' 

In [22]:
sido_correction = {
    '서울특별시': '서울특별시',
    '인천광역시': '인천광역시',
    '인천광역시시': '인천광역시',
    '대구광역시': '대구광역시',
    '대구광역시시': '대구광역시',
    '부산광역시': '부산광역시',
    '부산광역시시': '부산광역시',
    '경기도': '경기도',
    '강원도': '강원도',
    '충청북도': '충청북도',
    '충청남도': '충청남도',
    '전라북도': '전라북도',
    '전라남도': '전라남도',
    '제주특별자치도': '제주특별자치도',
    '세종특별자치시': '세종특별자치시',
    '경상북도': '경상북도',
    '경북': '경상북도',
    '경상남도': '경상남도',
    '광주광역시': '광주광역시',
    '광주': '광주광역시',
    '울산광역시': '울산광역시',
    '울산광역시시': '울산광역시',
    '대전광역시': '대전광역시',
    '대전광역시시': '대전광역시'
}

In [23]:
code['시도'] = code['시도'].map(sido_correction)
code['시도'].unique()

array(['강원도', '충청북도', '경기도', '서울특별시', '인천광역시', '충청남도', '경상북도', '대구광역시',
       '경상남도', '전라북도', '부산광역시', '울산광역시', '대전광역시', '세종특별자치시', '전라남도',
       '광주광역시', '제주특별자치도'], dtype=object)

In [24]:
code[code['시도']=='강원도']['시군구'].unique()

array(['평창군', '정선군', '태백시', '삼척시', '영월군', '횡성군', '원주시', '홍천군', '화천군',
       '양구군', '춘천시', '인제군', '가평군', '파주시', '철원군', '고성군', '양양군', '속초시',
       '강릉시', '동해시'], dtype=object)

In [25]:
codes = code[code['시도']=='강원도']['지점코드'].unique()
codes_list = list(codes)
code_Gangwon = ','.join(codes_list)
code_Gangwon

'1001A05,1001A10,1001A15,1001A20,1001A25,1001A30,1001A35,1001A40,1001A45,1001A50,1001A55,1001A60,1001A65,1001A70,1001A75,1001A80,1001A85,1001B10,1002A10,1002A20,1002A30,1002A40,1002A50,1002A55,1002D10,1002D20,1002D30,1002D40,1002D50,1002D60,1003A05,1003A10,1003A15,1005A55,1005A60,1005D10,1005D20,1006A05,1006A10,1006A15,1006A20,1006A25,1006A30,1006A35,1006A40,1006A45,1006A50,1006A55,1006A60,1006A65,1006A70,1006A80,1006B10,1006B20,1006B30,1006D05,1006D10,1006D15,1006D20,1006D25,1006D30,1006D35,1006D40,1006D45,1006D50,1006D55,1006D60,1006E11,1009A10,1009B10,1009B20,1010A10,1010A30,1010A40,1010A50,1010B10,1010B20,1010B30,1010B40,1010B50,1010B60,1010D10,1010D20,1011A10,1011A20,1011A30,1011D10,1012A20,1012A30,1012A40,1012B10,1012B20,1012B30,1012B35,1012B40,1012D10,1012E11,1013A05,1013A10,1013A20,1013A40,1013A50,1013A60,1013B10,1013B20,1013B30,1013D10,1013D20,1013D30,1014A10,1014A20,1014A30,1014A70,1014D10,1014D20,1014D30,1014D40,1014D50,1014D60,1014D70,1014D80,1014E10,1019D10,1021D10,1022A05

### 물환경 수질측정망 운영결과 월평균 DB

In [26]:
water_qual_url = 'http://apis.data.go.kr/1480523/WaterQualityService/getWaterMeasuringListMavg'
water_qual_params ={'serviceKey' : myKEY, 'pageNo' : '1', 'numOfRows' : '5000', 'resultType' : 'JSON', 'ptNoList' : code_Gangwon, 'wmyrList' : '2020'}

water_qual_response = requests.get(water_qual_url, params=water_qual_params)
#print(water_qual_response.content)

In [27]:
water_qual_result = json.loads(water_qual_response.content)
#water_qual_result['getWaterMeasuringListMavg']['item']

In [28]:
water_qual = pd.DataFrame(water_qual_result['getWaterMeasuringListMavg']['item'])
water_qual.sample()

,ROWNO,PTNM,PTNO,WMYR,WMOD,ITEMAMNT,ITEMTEMP,ITEMPH,ITEMDOC,ITEMBOD,ITEMCOD,ITEMSS,ITEMTCOLI,ITEMTN,ITEMTP,ITEMCD,ITEMCN,ITEMPB,ITEMCR6,ITEMAS,ITEMHG,ITEMCU,ITEMABS,ITEMPCB,ITEMOP,ITEMMN,ITEMTRANS,ITEMCLOA,ITEMZN,ITEMCR,ITEMFE,ITEMPHENOL,ITEMNHEX,ITEMEC,ITEMTCE,ITEMPCE,ITEMNO3N,ITEMNH3N,ITEMECOLI,ITEMPOP,ITEMDTN,ITEMDTP,ITEMFL,ITEMCOL,ITEMCCL4,ITEMDCM,ITEMBENZENE,ITEMCHCL3,ITEMTOC,ITEMDEHP,ITEMANTIMON,ITEMDIOX,ITEMHCHO,ITEMHCB,ITEMNI,ITEMBA,ITEMSE
1388,1389,북천,1011A10,2020,08,NaN,21.4,7.3,8.8,0.5,1.9,0.2,NaN,0.872,0.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,76.0,NaN,NaN,0.717,0.0,140.0,0.001,0.851,0.004,NaN,None,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [29]:
water_qual.columns

Index(['ROWNO', 'PTNM', 'PTNO', 'WMYR', 'WMOD', 'ITEMAMNT', 'ITEMTEMP',
       'ITEMPH', 'ITEMDOC', 'ITEMBOD', 'ITEMCOD', 'ITEMSS', 'ITEMTCOLI',
       'ITEMTN', 'ITEMTP', 'ITEMCD', 'ITEMCN', 'ITEMPB', 'ITEMCR6', 'ITEMAS',
       'ITEMHG', 'ITEMCU', 'ITEMABS', 'ITEMPCB', 'ITEMOP', 'ITEMMN',
       'ITEMTRANS', 'ITEMCLOA', 'ITEMZN', 'ITEMCR', 'ITEMFE', 'ITEMPHENOL',
       'ITEMNHEX', 'ITEMEC', 'ITEMTCE', 'ITEMPCE', 'ITEMNO3N', 'ITEMNH3N',
       'ITEMECOLI', 'ITEMPOP', 'ITEMDTN', 'ITEMDTP', 'ITEMFL', 'ITEMCOL',
       'ITEMCCL4', 'ITEMDCM', 'ITEMBENZENE', 'ITEMCHCL3', 'ITEMTOC',
       'ITEMDEHP', 'ITEMANTIMON', 'ITEMDIOX', 'ITEMHCHO', 'ITEMHCB', 'ITEMNI',
       'ITEMBA', 'ITEMSE'],
      dtype='object')

In [30]:
selected_columns = ['ROWNO', 'PTNM', 'PTNO', 'WMYR', 'WMOD', 'ITEMTCOLI']
water_qual = water_qual[selected_columns]
water_qual.sample()

,ROWNO,PTNM,PTNO,WMYR,WMOD,ITEMTCOLI
1317,1318,섬강5,1006A70,2020,08,NaN


In [31]:
new_columns = {
    'ROWNO': '행번호',
    'PTNO': '조사지점코드',
    'PTNM': '조사지점명',
    'WMYR': '측정년도',
    'WMOD': '측정월',
    'ITEMTCOLI': '측정값(총대장균군)'
}


In [32]:
# 컬럼 이름 변경
water_qual.rename(columns=new_columns, inplace=True)
water_qual.sample()

,행번호,조사지점명,조사지점코드,측정년도,측정월,측정값(총대장균군)
1231,1232,광포호,1301B20,2020,09,2880.0


In [33]:
# 결측치 수와 비율 계산
missing_count = water_qual.isnull().sum()
missing_ratio = round(missing_count / len(water_qual) * 100, 2)

# 결측치 수와 비율 출력
missing_info = pd.concat([missing_count, missing_ratio], axis=1, keys=['결측치 수', '결측치 비율 (%)'])
print(missing_info)

            결측치 수  결측치 비율 (%)
행번호             0         0.0
조사지점명           0         0.0
조사지점코드          0         0.0
측정년도            0         0.0
측정월             0         0.0
측정값(총대장균군)    175        12.3


In [34]:
from sklearn.impute import KNNImputer
import numpy as np

imputer = KNNImputer(n_neighbors=5)
water_qual[['측정값(총대장균군)']] = imputer.fit_transform(water_qual[['측정값(총대장균군)']])

In [35]:
water_qual[['측정값(총대장균군)']]

,측정값(총대장균군)
0,1892.000000
1,650.000000
2,1200.000000
3,373.000000
4,214.000000
...,...
1418,3688.653846
1419,3688.653846
1420,3688.653846
1421,3688.653846


연평균 데이터 제작

In [36]:
water_qual.to_csv('../data/water_quality_Gangwon.csv', index=False, encoding='utf-8-sig')